In [56]:
import os
import sys
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

import numpy as np
import ipywidgets as widgets # for the interactive user interface
from ipywidgets import *
from IPython.display import display, clear_output, SVG, HTML
from gvanim import Animation
from gvanim.jupyter import interactive

import graphviz as gv # for visualizing a tree using Digraph
from graphviz import Digraph, Graph
import pydot
import matplotlib.pyplot as plt
import pandas as pd
import random

from ipyfilechooser import FileChooser
import json
import time

import  tkinter as tk
from tkinter import filedialog
import shutil

import importlib.util

In [57]:
out, out1, out2, out3 = Output(), Output(), Output(), Output()
tab = Tab(children = [out1, out2, out3],
          layout=Layout(width='99%', height='auto'))
tab.set_title(0, 'Listas Enlazadas')
tab.set_title(1, 'Árboles Binarios')
tab.set_title(2, 'Grafos')

## Metodos de soporte

In [58]:
def create_n_random(n):
    nodos = list()
    for i in range(n*n):
        num = random.randint(1,50)
        if num not in nodos:
            nodos.append(num)  
        if len(nodos) == n:
            break
    return nodos 

In [59]:
def createTuples(nodos, tipo):
    tuples = list()
    if tipo == 1:
        ids = range(0,len(nodos)+2)
        tuples.append((ids[0],'First'))
        for i in range(1,len(ids)-1):
            tuples.append((ids[i],nodos[i-1]))
        if len(nodos) > 0:
            tuples.append((ids[len(nodos)+1],'None'))
    else:
        ids = range(0,len(nodos)+3)
        tuples.append((ids[0],'First'))
        if len(nodos) > 0:
            tuples.append((ids[1],'None'))
            for i in range(2,len(ids)-1):
                tuples.append((ids[i],nodos[i-2]))
            tuples.append((ids[len(nodos)+2],'None'))
    return tuples

## Metodos Graficos

In [60]:
def displayList(estructura, tipo, nodosX=list()):
    try:
        nodos = estructura.getNodeValues()
    except:
        e = '\tProblema en el método getNodeValues()'
        raise Exception(e)
    
    dot = Digraph()
    
    dot.graph_attr = {
        'rankdir': 'LR',
        'center': 'true',
        'size':'14,5',
        'ratio':'fill'
    }
    colorPointer = 'grey'
    colorRight = 'blue'
    colorLeft = 'green'
    colorHigh = 'red'
    
    tuples = createTuples(nodos, tipo)
    
    for i,j in tuples:
        if j == 'First' or j == 'None':
            dot.node(name=str(i), label=str(j), shape="square", color='white')
        elif j in nodosX:
            dot.node(name=str(i), label=str(j), shape="square", color=colorHigh)
        else:
            dot.node(name=str(i), label=str(j), shape="square")
    if len(nodos) == 0:
        dot.node(name='-1', label='', shape="square", color='white')
        dot.edge(str(tuples[0][0]), str('-1'), color=colorPointer)
    else:
        for i in range(1,len(tuples)):
            if tuples[i-1][1] == 'First' and tuples[i][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorPointer)
            elif tuples[i-1][1] == 'First' and tuples[i][1] == 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i+1][0]), color=colorPointer)
            elif tuples[i-1][1] != 'None':
                dot.edge(str(tuples[i-1][0]), str(tuples[i][0]), color=colorRight)
        if tipo == 2:
            for i in range(1, len(tuples)-1):
                if tuples[i][1] != 'None':
                    dot.edge(str(tuples[i][0]), str(tuples[i-1][0]), color = colorLeft)
    display(dot)


## Metodos de Enlace Lista Enlazada

In [61]:
def crearListaEnlazada(tipo, init, data={}): # falta la funcion de comparacion
    """
    Crea una lista enlazada

    Args:
        tipo: Sencilla o Doble
        init: Vacia o Random
    Returns:
        -
    Raises:
        Exception
    """
    
    if tipo == 1: txt = 'Sencilla - ' + init
    else: txt = 'Doble - ' + init
    
    txtNodos = ''
    try:
        estructura = listaEnlazada(tipo)
    except:
        e = "\tProblema al crear la lista enlazada, método listaEnlazada()"
        raise Exception(e)
    long = 0
    if init == 'Random':
        txtNodos = '\tValores: '
        long = random.randint(5,15)
        nodos = create_n_random(long)
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)
    elif init == 'Estática' or init == 'Archivo':
        try:
            txtNodos = '\tValores: '
            nodos = data["valores"]
            long = data["nodos"]
        except:
            raise Exception("El formato del archivo ingresado no es válido")
        for i in nodos:
            txtNodos = txtNodos + str(i) + ', '
            try:
                estructura.addNode_byValue(i)
            except:
                e = '\tProblema al añadir el elemento "'+str(i)+'", método addNode_byValue()'
                raise Exception(e)

    displayList(estructura, tipo) 
    print('Crear Lista Enlazada')
    print('\tLista Creada:', txt)
    print('\tNodos:', long)
    print(txtNodos[:-2])
    return estructura

def anadirNodoLista(estructura, tipo, nodo):
    """
    Añade un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        estructura.addNode_byValue(nodo)
    except:
        e = '\tProblema al añadir el elemento "'+str(nodo)+'", método addNode_byValue()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo)
    print('Añadir elemento\n\tElemento "'+str(nodo)+'" añadido exitosamente')
    
def eliminarNodoLista(estructura, tipo, nodo):
    """
    Elimina un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.deleteNode_byValue(nodo)
    except:
        e = '\tProblema al eliminar el elemento "'+str(nodo)+'", método deleteNode_byValue()'

    out.clear_output()
    displayList(estructura, tipo)
    if ans:
        print('Eliminar elemento\n\tEl elemento "'+ str(nodo)+ '" se ha eliminado exitosamente')
    else:
        print('Eliminar elemento\n\tEl elemento "' + str(nodo)+ '" no pertenece a la lista')

def encontrarNodoLista(estructura, tipo, nodo):
    """
    Encuentra un nodo en la estrcutura

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        ans = estructura.isNodeValue(nodo)
    except:
        e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
        raise Exception(e)
    out.clear_output()
    lista = list()
    lista.append(nodo)
    if ans:
        displayList(estructura, tipo, lista)
        print('¿Existe el elemento '+str(nodo)+'? \n\tEl elemento "' + str(nodo) + '" SI se encuentra en la lista')
    else:    
        displayList(estructura, tipo)
        print('¿Existe el elemento '+str(nodo)+'? \n\tEl elemento "' + str(nodo) + '" NO se encuentra en la lista')
        
def findAdjacentNodeLista(estructura, tipo, nodo):
    """
    Encuentra los adyacentes de un nodo a lista

    Args:
        estructura: lista enlazada
        nodo: valor del nodo
    Returns:
        -
    Raises:
        Exception
    """
    try:
        listaAdj = estructura.findAdjacentNode(nodo)
    except:
        e = '\tProblema al buscar los adyacentes del elemento "' + str(nodo) + '" , método findAdjacentNode()'
        raise Exception(e)
    
    out.clear_output()
    displayList(estructura, tipo, listaAdj)
    
    if len(listaAdj) == 0 :
        try:
            ans = estructura.isNodeValue(nodo)
        except:
            e = '\tProblema al buscar el elemento "' + str(nodo) + '", método isNodeValue()'
            raise Exception(e)
        if ans:
            txt = 'Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no tiene adyacentes'
        else:
            txt = 'Encontrar Adyacentes\n\tEl elemento "'+str(nodo)+ '" no existe en la lista'
        print(txt)
    else:
        txt = ''
        for i in listaAdj:
            txt = txt + str(i) + ', '
        txt2 = 'Encontrar Adyacentes\n\tEl elemento "'+ str(nodo)+ '" tiene '+str(len(listaAdj))+' adyacente(s): ' + txt[:-2]
        print(txt2)

def darTodosLosNodos(estructura, tipo):
    """
    Retorna una lista con todos los nodos de la lista

    Args:
        estructura: lista enlazada
    Returns:
        Lista de los elementos de la lista
    Raises:
        Exception
    """
    try:
        nodos = estructura.getNodeValues()
    except:
        raise Exception('\tProblema al obtener todos los elemento, método getNodeValues()')
    out.clear_output()
    displayList(estructura, tipo)
    txt = ''
    for i in nodos:
        txt = txt + str(i) + ', '
    
    print('Encontrar Todos')
    print('\tTotal Nodos:', str(len(nodos)))
    print('\tValores:', txt[:-2])


## Options Lista Enlazada

In [62]:
import this

listaType = widgets.RadioButtons(
    options = ['Sencilla', 'Doble'],
    disabled = False
)

listaInit = widgets.RadioButtons(
    options = ['Vacia', 'Random', 'Estática' ,'Archivo'],
    disabled = False
)

listaInputNode = widgets.Text(
    value = '',
    placeholder = 'Valor del nodo',
    disabled = False,
    layout = {'width': '140px', 'justify_content':'center'}
)

btn_crear = Button(description='Crear', width='extended')
def on_button_crear_clicked(b):
    with out:
        out.clear_output()
        if listaType.value == 'Sencilla':  this.type = 1
        else: this.type = 2
        try:
            if listaInit.value == 'Estática':
                try:
                    name = 'creation files/lista_enlazada_1.json'
                    json_data = open(name)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + name
                    raise Exception (e)               
                this.estructura = crearListaEnlazada(this.type, listaInit.value, data)
            elif listaInit.value == 'Archivo':
                try:
                    root = tk.Tk()
                    file_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Json File", "*.json")])
                    root.destroy()
                    json_data = open(file_path)
                    data = json.load(json_data)
                except:
                    e = "\tProblema al cargar el archivo " + file_path
                    raise Exception (e)
                this.estructura = crearListaEnlazada(this.type, listaInit.value, data)
            else:
                this.estructura = crearListaEnlazada(this.type, listaInit.value)
        except Exception as e:
            print('Hubo un problema al intentar crear la lista')
            print(e)
btn_crear.on_click(on_button_crear_clicked)

btn_addNode = Button(description='Añadir', width='extended')
def on_button_addNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                anadirNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar añadir un elemento')
                print(e)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                eliminarNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar eliminar un elemento')
                print(e)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Validar', width='extended')
def on_button_findNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                encontrarNodoLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar un elemento')
                print(e)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        value = listaInputNode.value.strip() 
        if len(value) > 0:
            try:
                try:
                    data = int(value)
                except:
                    data = value
                try:
                    this.estructura
                except:
                    raise Exception("\tLa estructura no ha sido creada")
                findAdjacentNodeLista(this.estructura, this.type, data)
                listaInputNode.value = ''
            except Exception as e:
                print('Hubo un problema al intentar encontrar los adyacentes de un elemento')
                print(e)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_todosNodos = Button(description='Encontrar Todos', width='extended')
def on_button_todosNodos_clicked(b):
    with out:    
        try:
            try:
                this.estructura
            except:
                raise Exception("\tLa estructura no ha sido creada")
            darTodosLosNodos(this.estructura, this.type)
        except Exception as e:
            print('Hubo un problema al intentar encontrar todos los nodos')
            print(e)
btn_todosNodos.on_click(on_button_todosNodos_clicked)

## Options Arbol Binario

In [63]:
btn_crearArbolOption = Button(description='Crear', width='extended')
def on_button_crearArbolOption_clicked(b):
    with out:
        print('Crear Arbol')
btn_crearArbolOption.on_click(on_button_crearArbolOption_clicked)

btn_addNodeArbolOption = Button(description='Añadir', width='extended')
def on_button_addNodeArbolOption_clicked(b):
    with out:
        print('Add node arbol')
btn_addNodeArbolOption.on_click(on_button_addNodeArbolOption_clicked)

btn_deleteNodeArbolOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeArbolOption_clicked(b):
    with out:
        print('delete node Arbol')
btn_deleteNodeArbolOption.on_click(on_button_deleteNodeArbolOption_clicked)

btn_findNodeArbolOption = Button(description='Validar', width='extended')
def on_button_findNodeArbolOption_clicked(b):
    with out:
        print('Find node Arbol')
btn_findNodeArbolOption.on_click(on_button_findNodeArbolOption_clicked)

btn_findAdjNodeArbolOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeArbolOption_clicked(b):
    with out:
        print('find adj Arbol')
btn_findAdjNodeArbolOption.on_click(on_button_findAdjNodeArbolOption_clicked)

## Options Grafo

In [64]:
btn_crearGrafoOption = Button(description='Crear', width='extended')
def on_button_crearGrafoOption_clicked(b):
    with out:
        print('Crear Grafo')
btn_crearGrafoOption.on_click(on_button_crearGrafoOption_clicked)

btn_addNodeGrafoOption = Button(description='Añadir', width='extended')
def on_button_addNodeGrafoOption_clicked(b):
    with out:
        print('Add node Grafo')
btn_addNodeGrafoOption.on_click(on_button_addNodeGrafoOption_clicked)

btn_deleteNodeGrafoOption = Button(description='Eliminar', width='extended')
def on_button_deleteNodeGrafoOption_clicked(b):
    with out:
        print('delete node Grafo')
btn_deleteNodeGrafoOption.on_click(on_button_deleteNodeGrafoOption_clicked)

btn_findNodeGrafoOption = Button(description='Validar', width='extended')
def on_button_findNodeGrafoOption_clicked(b):
    with out:
        print('Find node Grafo')
btn_findNodeGrafoOption.on_click(on_button_findNodeGrafoOption_clicked)

btn_findAdjNodeGrafoOption = Button(description='Adyacentes', width='extended')
def on_button_findAdjNodeGrafoOption_clicked(b):
    with out:
        print('find adj Grafo')
btn_findAdjNodeGrafoOption.on_click(on_button_findAdjNodeGrafoOption_clicked)

## Botones Lista Enlazada

In [65]:
form_item_layout = Layout(
    #display='flex',
    justify_content='center'
)

# Panel de Opciones Lista Enlazada
items_optionsCreateList = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Lista Enlazada')], layout=form_item_layout),
    Box([Label(value='Tipo:')]),
    Box([listaType]),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crear]),
]
items_optionsAddNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
]
items_optionsDeleteNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
]
items_optionsFindNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
]
items_optionsFindAdjNode = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
]
items_optionsFindAllNodes = [
    Box([Label(value=' ')]),
    #Box([Label(value='Encontrar Todos los Elementos')], layout=form_item_layout),
    Box([btn_todosNodos], layout=form_item_layout),
]

layoutOptionsCreate = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNode = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAllNodes = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateList = Box(items_optionsCreateList, layout=layoutOptionsCreate)
optionsAddNode = Box(items_optionsAddNode, layout=layoutOptionsAddNode)
optionsDeleteNode = Box(items_optionsDeleteNode, layout=layoutOptionsDeleteNode)
optionsFindNode = Box(items_optionsFindNode, layout=layoutOptionsFindNode)
optionsFindAdjNode = Box(items_optionsFindAdjNode, layout=layoutOptionsFindAdjNode)
optionsFindAllNodes = Box(items_optionsFindAllNodes, layout=layoutOptionsFindAllNodes)


# Panel de Opciones Arbol
items_optionsCreateArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Árbol Binario')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearArbolOption]),
]
items_optionsAddNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeArbolOption], layout=form_item_layout),
]
items_optionsDeleteNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeArbolOption], layout=form_item_layout),
]
items_optionsFindNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeArbolOption], layout=form_item_layout),
]
items_optionsFindAdjNodeArbol = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeArbolOption], layout=form_item_layout),
]

layoutOptionsCreateArbol = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeArbol = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateArbol = Box(items_optionsCreateArbol, layout=layoutOptionsCreateArbol)
optionsAddNodeArbol = Box(items_optionsAddNodeArbol, layout=layoutOptionsAddNodeArbol)
optionsDeleteNodeArbol = Box(items_optionsDeleteNodeArbol, layout=layoutOptionsDeleteNodeArbol)
optionsFindNodeArbol = Box(items_optionsFindNodeArbol, layout=layoutOptionsFindNodeArbol)
optionsFindAdjNodeArbol = Box(items_optionsFindAdjNodeArbol, layout=layoutOptionsFindAdjNodeArbol)

# Panel de Opciones Grafo
items_optionsCreateGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Crear Grafo')], layout=form_item_layout),
    Box([Label(value='Inicialización:')]),
    Box([listaInit]),
    Box([btn_crearGrafoOption]),
]
items_optionsAddNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Añadir Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_addNodeGrafoOption], layout=form_item_layout),
]
items_optionsDeleteNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Eliminar Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_deleteNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Existe Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findNodeGrafoOption], layout=form_item_layout),
]
items_optionsFindAdjNodeGrafo = [
    Box([Label(value=' ')]),
    Box([Label(value='Adyacentes Elemento')], layout=form_item_layout),
    Box([listaInputNode], layout=form_item_layout),
    Box([btn_findAdjNodeGrafoOption], layout=form_item_layout),
]

layoutOptionsCreateGrafo = Layout(
    display='none',
    #border = 'solid 1px',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsAddNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsDeleteNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)
layoutOptionsFindAdjNodeGrafo = Layout(
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
)

optionsCreateGrafo = Box(items_optionsCreateGrafo, layout=layoutOptionsCreateGrafo)
optionsAddNodeGrafo = Box(items_optionsAddNodeGrafo, layout=layoutOptionsAddNodeGrafo)
optionsDeleteNodeGrafo = Box(items_optionsDeleteNodeGrafo, layout=layoutOptionsDeleteNodeGrafo)
optionsFindNodeGrafo = Box(items_optionsFindNodeGrafo, layout=layoutOptionsFindNodeGrafo)
optionsFindAdjNodeGrafo = Box(items_optionsFindAdjNodeGrafo, layout=layoutOptionsFindAdjNodeGrafo)

allOptions = [optionsCreateList,
            optionsAddNode,
            optionsDeleteNode,
            optionsFindNode,
            optionsFindAdjNode,
            optionsFindAllNodes,
            optionsCreateArbol,
            optionsAddNodeArbol,
            optionsDeleteNodeArbol,
            optionsFindNodeArbol,
            optionsFindAdjNodeArbol,
            optionsCreateGrafo,
            optionsAddNodeGrafo,
            optionsDeleteNodeGrafo,
            optionsFindNodeGrafo,
            optionsFindAdjNodeGrafo]

def hideShowOptions(option):
    for i in allOptions:
        i.layout.visibility = 'hidden'
        i.layout.display = 'none'
    listaInputNode.value = ''
       
    option.layout.display = 'block'
    option.layout.visibility = "visible"

# Panel de Opciones Lista Enlazada
btn_crearLista = Button(description='Crear Lista Enlazada', width='extended')
def on_button_crearLista_clicked(b):
    with out:
        hideShowOptions(optionsCreateList)
btn_crearLista.on_click(on_button_crearLista_clicked)

btn_addNode = Button(description='Añadir Elemento', width='extended')
def on_button_addNode_clicked(b):
    with out:
        hideShowOptions(optionsAddNode)
btn_addNode.on_click(on_button_addNode_clicked)

btn_deleteNode = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNode_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNode)
btn_deleteNode.on_click(on_button_deleteNode_clicked)

btn_findNode = Button(description='Existe Elemento', width='extended')
def on_button_findNode_clicked(b):
    with out:
        hideShowOptions(optionsFindNode)
btn_findNode.on_click(on_button_findNode_clicked)

btn_findAdjNode = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNode_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNode)
btn_findAdjNode.on_click(on_button_findAdjNode_clicked)

btn_findAllNodes = Button(description='Encontrar Todos', width='extended')
def on_button_findAllNodes_clicked(b):
    with out:
        hideShowOptions(optionsFindAllNodes)
btn_findAllNodes.on_click(on_button_findAllNodes_clicked)

# Panel de Opciones Arbol
btn_crearArbol = Button(description='Crear Árbol Binario', width='extended')
def on_button_crearArbol_clicked(b):
    with out:
        hideShowOptions(optionsCreateArbol)
btn_crearArbol.on_click(on_button_crearArbol_clicked)

btn_addNodeArbol = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeArbol)
btn_addNodeArbol.on_click(on_button_addNodeArbol_clicked)

btn_deleteNodeArbol = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeArbol)
btn_deleteNodeArbol.on_click(on_button_deleteNodeArbol_clicked)

btn_findNodeArbol = Button(description='Existe Elemento', width='extended')
def on_button_findNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeArbol)
btn_findNodeArbol.on_click(on_button_findNodeArbol_clicked)

btn_findAdjNodeArbol = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeArbol_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeArbol)
btn_findAdjNodeArbol.on_click(on_button_findAdjNodeArbol_clicked)

# Panel de Opciones Grafo
btn_crearGrafo = Button(description='Crear Grafo', width='extended')
def on_button_crearGrafo_clicked(b):
    with out:
        hideShowOptions(optionsCreateGrafo)
btn_crearGrafo.on_click(on_button_crearGrafo_clicked)

btn_addNodeGrafo = Button(description='Añadir Elemento', width='extended')
def on_button_addNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsAddNodeGrafo)
btn_addNodeGrafo.on_click(on_button_addNodeGrafo_clicked)

btn_deleteNodeGrafo = Button(description='Eliminar Elemento', width='extended')
def on_button_deleteNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsDeleteNodeGrafo)
btn_deleteNodeGrafo.on_click(on_button_deleteNodeGrafo_clicked)

btn_findNodeGrafo = Button(description='Existe Elemento', width='extended')
def on_button_findNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindNodeGrafo)
btn_findNodeGrafo.on_click(on_button_findNodeGrafo_clicked)

btn_findAdjNodeGrafo = Button(description='Encontrar Adyacentes', width='extended')
def on_button_findAdjNodeGrafo_clicked(b):
    with out:
        hideShowOptions(optionsFindAdjNodeGrafo)
btn_findAdjNodeGrafo.on_click(on_button_findAdjNodeGrafo_clicked)

In [66]:
# Layouts
layout_canvas = Layout(height='550px', border='1px dotted blue', overflow ='auto')
layout_options = Layout(height='auto', width='auto')#, border='1px dotted blue')

buttons_listas = [
    Box([btn_crearLista], layout=form_item_layout),
    Box([btn_addNode], layout=form_item_layout),
    Box([btn_deleteNode], layout=form_item_layout),
    Box([btn_findNode], layout=form_item_layout),
    Box([btn_findAdjNode], layout=form_item_layout),
    Box([btn_findAllNodes], layout=form_item_layout),
]
buttons_arboles = [
    Box([btn_crearArbol], layout=form_item_layout),
    Box([btn_addNodeArbol], layout=form_item_layout),
    Box([btn_deleteNodeArbol], layout=form_item_layout),
    Box([btn_findNodeArbol], layout=form_item_layout),
    Box([btn_findAdjNodeArbol], layout=form_item_layout),
]
buttons_grafos = [
    Box([btn_crearGrafo], layout=form_item_layout),
    Box([btn_addNodeGrafo], layout=form_item_layout),
    Box([btn_deleteNodeGrafo], layout=form_item_layout),
    Box([btn_findNodeGrafo], layout=form_item_layout),
    Box([btn_findAdjNodeGrafo], layout=form_item_layout),
]

# Cargar Estructura
btn_cargarEstructura = Button(description='Cargar Estructura', width='extended')
def on_button_cargarEstructura_clicked(b):
    with out:       
        try:
            root = tk.Tk()
            src_path = tk.filedialog.askopenfilename(title='Select a file...',filetypes=[("Python File", "*.py")])
            root.destroy()
            spec=importlib.util.spec_from_file_location("listaEnlazada",src_path)
            foo = importlib.util.module_from_spec(spec)
  
            spec.loader.exec_module(foo)
            
            foo.listaEnlazada(1).example()
            this.file = foo
        except:
            e = "\tProblema al cargar el archivo " + src_path
            raise Exception (e)
        buttonsListas.layout.visibility = 'visible'
        buttonsListas.layout.display = 'block'
btn_cargarEstructura.on_click(on_button_cargarEstructura_clicked)

buttons_crearEstructuras = [
    Box([btn_cargarEstructura], layout=form_item_layout),
]


buttonsCrear = Box(buttons_crearEstructuras, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsListas = Box(buttons_listas, layout=Layout(
    #display='flex',
    display='none',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsArboles = Box(buttons_arboles, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))
buttonsGrafos = Box(buttons_grafos, layout=Layout(
    display='flex',
    flex_flow='column',
    align_items='stretch',
    width='auto',
    justify_content='center'
))

canvas = HBox([out], layout=layout_canvas)
optionsListas = VBox([buttonsCrear,
                      buttonsListas,
                      optionsCreateList,
                      optionsAddNode,
                      optionsDeleteNode,
                      optionsFindNode,
                      optionsFindAdjNode,
                      optionsFindAllNodes],
                      layout=layout_options)
optionsArboles = VBox([buttonsArboles,
                        optionsCreateArbol,
                        optionsAddNodeArbol,
                        optionsDeleteNodeArbol,
                        optionsFindNodeArbol,
                        optionsFindAdjNodeArbol],
                        layout=layout_options)
optionsGrafos = VBox([buttonsGrafos,            
                    optionsCreateGrafo,
                    optionsAddNodeGrafo,
                    optionsDeleteNodeGrafo,
                    optionsFindNodeGrafo,
                    optionsFindAdjNodeGrafo],
                    layout=layout_options)

with out1:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsListas,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)
with out2:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsArboles,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)
with out3:
    app = AppLayout(header=None,
          left_sidebar=None,
          center=canvas,
          right_sidebar=optionsGrafos,
          pane_widths=['0%', '80%', '20%'],
          pane_heights=['0%','550px',1],
          footer=None)
    display(app)

display(tab)